In [5]:
from pipeline import Pipeline
from sklearn.model_selection import train_test_split
import gc


In [6]:
d = Pipeline(model_type="predict_upgrade")
d.concat_df_with_oh_encoding()
df = (
    d.df
)

2022-10-12 22:42:53.956 | INFO     | pipeline:read_all_files:43 - loaded booking data
2022-10-12 22:42:59.708 | INFO     | pipeline:read_all_files:45 - loaded ticket data
2022-10-12 22:43:20.349 | INFO     | pipeline:read_all_files:47 - loaded flight coupon data
2022-10-12 22:43:20.350 | INFO     | pipeline:merge_files:62 - 2022-10-12 22:43:20.350556 merging datasets
2022-10-12 22:43:46.882 | INFO     | pipeline:clean_df:293 - 2022-10-12 22:43:46.882859 cleaning df ...
2022-10-12 22:44:19.598 | INFO     | pipeline:convert_datetime_columns_to_pandas_format:161 - converted datetime columns to pandas format
2022-10-12 22:44:23.034 | INFO     | pipeline:filter_wrong_ticket_prices:144 - filtered wrong ticket prices
2022-10-12 22:44:26.092 | INFO     | pipeline:filter_wrong_booking_window:165 - filtered wrong booking windows
2022-10-12 22:44:29.363 | INFO     | pipeline:filter_nonpositive_flight_distance:169 - filtered nonpositive flight distance
2022-10-12 22:45:38.271 | INFO     | pipeline

In [7]:
df

,COUPON_NUMBER,TIME_DEPARTURE_LOCAL_TIME,FLIGHT_DISTANCE,UPGRADED_FLAG,TOTAL_PRICE_PLN,PAX_GENDER,CORPORATE_CONTRACT_FLG,LOYAL_CUSTOMER,BOOKING_WINDOW_D,STAY_LENGTH_D,...,VAB_STANDARD,PAX_TYPE_CHILD,PAX_TYPE_INFANT,SALES_CHANNEL_ATO_CTO,SALES_CHANNEL_CALL CENTER,SALES_CHANNEL_DCS,SALES_CHANNEL_LOT TRAVEL,SALES_CHANNEL_LOT.COM,TRIP_TYPE_ONE WAY,TRIP_TYPE_ROUND TRIP
0,2,19,1344,0,620.35,1,0,0,121.0,4.0,...,1,0,0,0,0,0,1,0,0,1
1,1,13,6942,0,2151.49,1,0,0,228.0,21.0,...,0,0,0,0,0,0,0,0,0,1
2,2,16,7521,0,2270.18,0,0,0,55.0,15.0,...,0,0,0,0,0,0,0,1,0,1
3,1,6,721,0,1096.16,0,0,0,12.0,7.0,...,1,0,0,0,0,0,0,0,0,1
4,2,18,522,0,752.15,1,0,0,6.0,1.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9224582,1,19,523,0,813.64,1,0,0,0.0,-9999.0,...,1,0,0,0,0,0,0,0,1,0
9224583,2,17,1070,0,675.51,1,0,0,116.0,8.0,...,0,0,0,0,0,0,0,1,0,1
9224584,1,7,753,0,1408.80,1,0,0,36.0,-9999.0,...,0,0,0,0,0,0,0,0,1,0
9224585,1,18,940,0,1157.97,1,0,0,37.0,3.0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
df.dropna(inplace=True)
X_train, X_test, y_train, y_test = train_test_split(
        df.drop(['UPGRADED_FLAG'], axis=1), df.iloc[:, 3], test_size=0.2, random_state=42
    )
del(df)
gc.collect()

0

In [13]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(
    scale_pos_weight=1046.6532651902328,
    seed=42,
    use_label_encoder=False,
    tree_method="gpu_hist",
    gpu_id=0,
)
xgb_model.fit(X_train[['FLIGHT_DISTANCE']], y_train, eval_metric="aucpr", verbose=2)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=0, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1046.6532651902328,
              seed=42, subsample=1, tree_method='gpu_hist',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [14]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
import warnings
warnings.simplefilter(action='ignore')
kfold2 = StratifiedKFold(n_splits=5)
res = cross_val_score(xgb_model, X_test[['FLIGHT_DISTANCE']], y_test, cv=kfold2, scoring='roc_auc', fit_params={'eval_metric':'aucpr'})
print("średnia:", res.mean())
print('auc score:', res)



średnia: 0.9414075131979891
auc score: [0.93999375 0.93940649 0.94246738 0.9420799  0.94309005]


In [35]:
import numpy as np
results= res
best_result=results.mean()

In [36]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
import xgboost as xgb
import warnings

warnings.simplefilter(action="ignore")
lista_kolumn = ["FLIGHT_DISTANCE"]
wszystkie_kolumny = X_test.columns.tolist()
x = 0
best_col=''
while best_col != None:
    best_col = None
    while x < len(wszystkie_kolumny):
        print(x, wszystkie_kolumny[x])
        while wszystkie_kolumny[x] in lista_kolumn:
            x += 1
        else:
            lista_kolumn.append(wszystkie_kolumny[x])
        kolumny = X_test[lista_kolumn]
        xgb_model = xgb.XGBClassifier(
            scale_pos_weight=1046.6532651902328,
            seed=42,
            use_label_encoder=False,
            tree_method="gpu_hist",
            gpu_id=0,
        )
        kfold = StratifiedKFold(n_splits=5)
        results = cross_val_score(
            xgb_model,
            kolumny,
            y_test,
            cv=kfold,
            scoring="roc_auc",
            fit_params={"eval_metric": "aucpr"},
        )
        print("średnia:", results.mean())
        print("auc score:", results)
        print("kolumny:", kolumny.columns)
        if best_result < results.mean():
            best_col = wszystkie_kolumny[x]
            best_result = results.mean()
        lista_kolumn.remove(wszystkie_kolumny[x])
        x += 1
        print("best column: ", best_col, best_result)
    if best_col: lista_kolumn.append(best_col)


0 COUPON_NUMBER
średnia: 0.9430191916574671
auc score: [0.94133795 0.94048223 0.94487507 0.94475615 0.94364456]
kolumny: Index(['FLIGHT_DISTANCE', 'COUPON_NUMBER'], dtype='object')
best column:  COUPON_NUMBER 0.9430191916574671
1 TIME_DEPARTURE_LOCAL_TIME
średnia: 0.9397240565687917
auc score: [0.93900204 0.93775087 0.93987766 0.94229562 0.9396941 ]
kolumny: Index(['FLIGHT_DISTANCE', 'TIME_DEPARTURE_LOCAL_TIME'], dtype='object')
best column:  COUPON_NUMBER 0.9430191916574671
2 FLIGHT_DISTANCE
średnia: 0.9491997086641752
auc score: [0.94711454 0.94486991 0.95245629 0.94955179 0.95200601]
kolumny: Index(['FLIGHT_DISTANCE', 'TOTAL_PRICE_PLN'], dtype='object')
best column:  TOTAL_PRICE_PLN 0.9491997086641752
4 PAX_GENDER
średnia: 0.941546310240226
auc score: [0.93909841 0.94051083 0.94340043 0.94208009 0.94264179]
kolumny: Index(['FLIGHT_DISTANCE', 'PAX_GENDER'], dtype='object')
best column:  TOTAL_PRICE_PLN 0.9491997086641752
5 CORPORATE_CONTRACT_FLG
średnia: 0.9410500912224993
auc score:

KeyboardInterrupt: 